In [1]:
# Implement DSA. I'll use the provided parameters

q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
p_hex = '800000000000000089e1855218a0e7dac38136ffafa72eda7' \
        '859f2171e25e65eac698c1702578b07dc2a1076da241c76c6' \
        '2d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebe' \
        'ac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2' \
        'b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc87' \
        '1a584471bb1'
g_hex = '5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119' \
        '458fef538b8fa4046c8db53039db620c094c9fa077ef389b5' \
        '322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a047' \
        '0f5b64c36b625a097f1651fe775323556fe00b3608c887892' \
        '878480e99041be601a62166ca6894bdd41a7054ec89f756ba' \
        '9fc95302291'

p = int(p_hex, base = 16)
g = int(g_hex, base = 16)

N = 40*4
L = len(p_hex)*4
assert (p-1) % q == 0

In [2]:
# Now implement the algorithm
# I'm largely following wikipedia https://en.wikipedia.org/wiki/Digital_Signature_Algorithm

from hashlib import sha256

# Create a hash function with |H| = N bits output
def Hash(x: bytes, N = N):
    assert type(x) is bytes
    assert N <= 256
    digest_bytes = sha256(x).digest()
    digest_int = int.from_bytes(digest_bytes, byteorder = 'big')
    return digest_int & ( ( 0b1 << N ) - 1 )

assert Hash(b'abc') < 2**N

In [3]:
from random import randint

def DSAGeneratePrivateKey(p = p, q = q, g = g):
    return randint(1, q-1)

def DSAGeneratePublicKey(x, p = p, q = q, g = g ):
    assert type(x) is int and 1 <= x <= q-1
    return pow(g, x, p)

def DSASign(message, x, p = p, q = q, g = g, H = Hash, leaky = False):
    # x is the privkey
    # If leaky, this will return the value of k, which can be used in an attack.
    # Don't be leaky!  That's only for demonstrating the attack.
    assert type(message) is bytes
    assert type(x) is int and 1 <= x <= q-1
    
    k = randint(1, q-1)
    r = pow(g, k, p) % q
    
    k_inv = pow(k, q-2, q) # https://en.wikipedia.org/wiki/Fermat%27s_little_theorem
    assert (k*k_inv) % q == 1
    s = ( k_inv*( H(message) + x*r  ) ) % q
    
    assert s != 0 and r != 0
    signature = dict(r = r, s = s)
    signature.update(k = k) if leaky else None
    return signature
    

def DSAVerify(message, r, s, y, p = p, q = q, g = g, H = Hash):
    # y is the pubkey
    assert type(message) is bytes
    assert 0 < r < q 
    assert 0 < s < q
    
    w  = pow(s, q-2, q) # w is s inverse, again using FLT
    assert w*s % q == 1
    u1 = w*H(message) % q
    u2 = w*r % q
    
    # Using property (A * B) mod C = (A mod C * B mod C) mod C
    v  = pow(g, u1, p) * pow(y, u2, p)
    v %= p
    v %= q
    
    return True if v == r else False

In [4]:
# Now test it

# Alice generates a keypair and signs a message
message = b'Hello, world!'
privkey = DSAGeneratePrivateKey()
pubkey = DSAGeneratePublicKey(privkey)
signature = DSASign(message, privkey)

# Bob verifies the message using the signature and Alice's pubkey
assert DSAVerify(message, y = pubkey, **signature)

In [11]:
# Ok now practice private key recovery from k

def AttackPrivateKey(r, s, k, message):
    r_inv = pow(r, q-2, q) # FLT
    x = ( ( (s*k) - Hash(message) ) * r_inv ) % q
    return x

signature = DSASign(message, privkey, leaky = True)
x = AttackPrivateKey(**signature, message = message)
assert x == privkey

In [20]:
# Ok, now practice using this on a version of DSASign that doesn't leak k...
# But does have the flaw that it chooses k from a narrow range

def DSASignBroken(message, x, p = p, q = q, g = g, H = Hash):
    # x is the privkey
    assert type(message) is bytes
    assert type(x) is int and 1 <= x <= q-1
    
    k = randint(1, 2**16-1)
    r = pow(g, k, p) % q
    
    k_inv = pow(k, q-2, q) # https://en.wikipedia.org/wiki/Fermat%27s_little_theorem
    assert (k*k_inv) % q == 1
    s = ( k_inv*( H(message) + x*r  ) ) % q
    
    assert s != 0 and r != 0
    signature = dict(r = r, s = s)
    return signature

def DSASignWithk(message, x, k, p = p, q = q, g = g, H = Hash):
    # x is the privkey
    assert type(message) is bytes
    assert type(x) is int and 1 <= x <= q-1
    
    r = pow(g, k, p) % q
    
    k_inv = pow(k, q-2, q) # https://en.wikipedia.org/wiki/Fermat%27s_little_theorem
    # assert (k*k_inv) % q == 1
    s = ( k_inv*( H(message) + x*r  ) ) % q
    
    # assert s != 0 and r != 0
    signature = dict(r = r, s = s)
    return signature

signature = DSASignBroken(message, privkey)

In [21]:
# Now as the attacker I can recover the privkey by force

for k in range(2**16):
    privkey_guess = AttackPrivateKey(**signature, k = k, message = message)
    if signature == DSASignWithk(message, privkey_guess, k = k):
        assert privkey_guess == privkey
        print('privkey =', privkey_guess)
        break

privkey = 303744735491515564103918382730084723110655543882
